# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
!pip install langchain-groq crewai crewai_tools langchain_community



In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

**Note**:
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
import os
from getpass import getpass
os.environ["GROQ_API_KEY"] = getpass("Enter GROQ_API_KEY: ")
os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_KEY: ")


Enter GROQ_API_KEY: ··········
Enter OPENAI_KEY: ··········


## crewAI Tools

In [ ]:
from crewai_tools import (
  PDFSearchTool,
)


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [ ]:
!pip install litellm

In [ ]:
llm_groq = ChatGroq(
    temperature=0,
    model_name="mixtral-8x7b-32768",
)

In [ ]:
Phillip_pdf_url = "/content/Phillip Censored.pdf"
Phillip_pdf_search_tool = PDFSearchTool(pdf=Phillip_pdf_url, query="operations on 11/08/2024",  description="Search for information regarding operations on 11/08/2024 in Phillip Capital reports.")

ABN_pdf_url = "/content/ABN_censor_2.pdf"
ABN_pdf_search_tool = PDFSearchTool(pdf=ABN_pdf_url, query="operations on 11/08/2024",description="Search for information regarding operations on 11/08/2024 in ABN reports.")

In [ ]:

trader1 = Agent(
    role="Commodities Trader",
    goal="Make sure to do thorough analysis on the PDF file "
         "reporting about our financial operations in the Phillip broker"
         "And specifically report the operations from the inputed date",
    tools=[Phillip_pdf_search_tool],
    verbose=True,
    backstory=(
        "As a Commodities Trader, your prowess in "
        "navigating and extracting specific requested "
        "information from pdf files (reports) is unmatched. "
        "You are a specialist in Phillip Capital Inc., ABN, Marex, and others "
        "and you know they are regulated by the U.S. Securities and Exchange Commission and the Commodity Futures Trading Commission. "
        "You know how to read their reports and extract every needed information, "
        "especially the information regarding volume and price of operations done on the specified date: {date}."
    ),
     allow_delegation=False,
)

In [ ]:

trader2 = Agent(
    role="Commodities Trader 2",
    goal="Make sure to do thorough analysis on the PDF file "
         "reporting about our financial operations in the ABN broker"
         "Specifically report the operations from the inputed date",
    tools=[ABN_pdf_search_tool],
    verbose=True,
    backstory=(
        "As a Commodities Trader, your prowess in "
        "navigating and extracting specific requested "
        "information from pdf files (reports) is unmatched. "
        "You are a specialist in Phillip Capital Inc., ABN, Marex, and others "
        "and you know they are regulated by the U.S. Securities and Exchange Commission and the Commodity Futures Trading Commission. "
        "You know how to read their reports and extract every needed information, "
        "especially the information regarding volume and price of operations done on the specified date: {date}."
    ),
     allow_delegation=False,
)

## Creating Tasks

In [ ]:
!pip install litellm

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
Phillip_task = Task(
    description=(
        "Analyze the pdf of the analisis report provided from your broker"
        "to extract information about the operations of that inputed day "
        "Use the tools to gather the text and identify "
        "and summarize the operations"
        "each tool corresponds to a different stockbroker"
        "The pdf file provided may contain"
        "various text elements, so it's crucial to capture all readable text"
    ),
    expected_output=(
        " A string containing the full analysis based on the text extracted from the pdf file. "
        "Mainly a text summary about the operations realized on that day"
    ),
    agent=trader1,
)

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
ABN_task = Task(
    description=(
        "Analyze the pdf of the analisis report provided from your broker"
        "to extract information about the operations of that inputed day "
        "Use the tools to gather the text and identify "
        "and summarize the operations"
        "each tool corresponds to a different stockbroker"
        "The pdf file provided may contain"
        "various text elements, so it's crucial to capture all readable text"
    ),
    expected_output=(
        " A string containing the full analysis based on the text extracted from the pdf file. "
        "Mainly a text summary about the operations realized on that day"
    ),
    agent=trader2,
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

## Creating the Crew

In [ ]:
job_application_crew = Crew(
    agents=[trader1, trader2],

    tasks=[Phillip_task, ABN_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
job_application_inputs = {
    'date': "11/08/2024"
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
import litellm
litellm.set_verbose = True

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Commodities Trader
## Task: Analyze the pdf of the analisis report provided from your brokerto extract information about the operations of that inputed day Use the tools to gather the text and identify and summarize the operationseach tool corresponds to a different stockbrokerThe pdf file provided may containvarious text elements, so it's crucial to capture all readable text
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FixedPDFSearchToolSchema
query
  Input should be a valid string [type=string_type, input_value={'description': 'Operatio...08/2024', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}}
Tool Description: A tool that 

- Dislplay the generated `tailored_resume.md` file.